# Markov Chain


## Definition
1. 각 노드는 여러개의 state 를 가질 수 있는 Random variable 이다. 
1. 각 링크는 전이 확률을 갖는다. 
  - ex. $P(z_j|z_i)=
  \left[ {\begin{array}{cc}
   0.7 & 0.2 & 0.1 \\
   0.2 & 0.3 & 0.5\\
   0.4 & 0.2 & 0.4\\
  \end{array} } \right]$

## Properties
1. Accessible
  - accessible : i 에서 j로 갈 수 있다. 
  - communicate : 양방향으로 갈 수 있다. 
1. Reducibility
  - communicate 하고 i 도 j도  S 안에 있으면 irreducible. 
  - 지속 적으로 이동 가능함. 
1. Periodicity
  - 어떤 주기를 갖고 state i 에 방문함.
  - 주기가 1이면 aperiodic 이라고 본다.
1. Transience
  - Transient 하지 않으면 recurrent : 어떤 state 가 다시 일어날 수 있다. (반드시 다시 등장한다.)
  - Transient 하면 그 state가 다시 등장하지 않는다. 
1. Ergodicity
  - recurrent + aperiodic = ergodic.
  - 모든 state가 ergodic 하면 markov chain 이 ergodic 하다고 한다.

## Stationary Distribution

1. Return time
  - $RT_i = \min{\{n >0 : X_n = i | X_0 = i}\}$
  - state i 에 방문한 후에 다시 i 에 방문하는데까지 걸리는 최소 시간
1. Limit theorem of Markov chain
  - 만약 Markov chain 이 irreducible and ergodic 하다면, 
  - $\pi_i $ : 어떤 특정 state 에 이 시스템이 있을 확률 분포 == stationary distribution 
  - $\pi T = \pi = \frac{1}{E[RT]}$
  
1. 다음 공식을 통해 Transition Matrix 만 있으면 stationary distribution 을 구할 수 있다.

1. reversible MC $\rightarrow$ $\pi$ is a stationary distribution 
  - $\pi_i T_{i, j} = \pi_j T_{j, i}$

## Markov Chain for Sampling

1. Importance sampling, rejection sampling, forward sampling 의 문제는 모든 샘플링이 독립이라고 보기 때문에, 이전 레코드들을 사용하지 않는다는 것이다.
1. 기존 정보를 활용해서 그 다음 Z 의 assignment 을 하겠다. 


## MC vs Markov Chain Monte Carlo

1. MC
  1. Transition rule 이 주어지고, stationary distribution 을 찾는 것이 주된 태스크였다.

1. MCMC
  1. Target stationary distribution $\pi$ 가 이미 알려져있다고 생각한다.
  1. 이걸 만드는 Transition rule 은 무엇인가에 관심이 있다.
  1. 이 Transition matrix 로부터 계속 샘플링 해서 원하는 stationary distribution 이 되는지 보면서 transition matrix 의 값을 바꿔나간다. 


## Conjugate Prior
- Likelihood and prior multiplication results in the prior distribution.
- Dirichlet distribution is the conjuget prior of the multinomial distribution.
- Enables sum to one technique.

## Markov Chain Example



In [210]:
import numpy as np
import random as rm

trans_matrix = np.array([[0.2,0.6,0.2],
                         [0.1,0.6,0.3],
                         [0.2,0.7,0.1]])

In [201]:
def transient(timestep, start_state, trans_matrix):
    num_of_state = len(trans_matrix)
    state = start_state
    for i in range(timestep):
        state = np.random.choice(num_of_state, p=trans_matrix[state])
    return state


def get_prob_by_montecarlo(start_state, end_state, timestep, trial=10000):
    count = 0.0
    for i in range(trial):
        last_state = transient(timestep, start_state, trans_matrix)
        if end_state == last_state:
            count += 1
    return count / trial


def get_prob_by_calc(start_state, end_state, timestep):
    prob = mat_pow(trans_matrix, timestep)[start_state, end_state]
    return prob


def mat_pow(A, num):
    ret = A
    for i in range(num-1):
        ret = np.matmul(ret, trans_matrix)
    return ret

ss -> sr = 0.2 * 0.2   
si -> ir = 0.6 * 0.3   
sr -> rr = 0.2 * 0.1   

In [160]:
0.2 * 0.2 + 0.6 * 0.3 + 0.2 * 0.1

0.24

In [220]:
# Q1.
start_state = 0
end_state = 2
timestep = 2

get_prob_by_montecarlo(start_state, end_state, timestep, trial=100000),\
get_prob_by_calc(start_state, end_state, timestep)

(0.24023, 0.24)

In [221]:
# Q2.
start_state = 1
end_state = 0
timestep = 10

get_prob_by_montecarlo(start_state, end_state, timestep, trial=20000),\
get_prob_by_calc(start_state, end_state, timestep)

(0.1392, 0.13761468049999998)

In [224]:
# Q3.
start_state = 0
end_state = 0
timestep = 15

get_prob_by_montecarlo(start_state, end_state, timestep, trial=20000),\
get_prob_by_calc(start_state, end_state, timestep)

(0.13405, 0.13761467889913798)

In [222]:
# Q3.
start_state = 1
end_state = 0
timestep = 15

get_prob_by_montecarlo(start_state, end_state, timestep, trial=20000),\
get_prob_by_calc(start_state, end_state, timestep)

(0.1349, 0.13761467889890497)

In [223]:
# Q3.
start_state = 2
end_state = 0
timestep = 15

get_prob_by_montecarlo(start_state, end_state, timestep, trial=20000),\
get_prob_by_calc(start_state, end_state, timestep)

(0.1352, 0.13761467889951495)

In [374]:
# Q3. 
evalues, evectors = np.linalg.eig(trans_matrix.T)
for idx, ev in enumerate(evalues):
    if np.isclose(ev, 1.0):
        break
print(np.matmul(evectors[:, idx], trans_matrix), evalues[idx] * evectors[:, idx])
prob = evectors[:, idx]/np.sum(evectors[:, idx])
print(prob[0])

[-0.20180184 -0.914835   -0.34978985] [-0.20180184 -0.914835   -0.34978985]
0.13761467889908252
